In [41]:
%load_ext autoreload
%autoreload 2

import plotly
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import formulas
import pandas as pd
import random
import dataframes

plotly.offline.init_notebook_mode(connected=True)
# pd.set_option("display.max_rows", None, "display.max_columns", None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Inputs
Assume fixed rate mortgage

In [42]:
# Loan
DOWN_PAYMENT = 200000
PRINCIPAL = 900000 # This is the value of house. Principal is probably a bad name for this.
LOAN_APR = 0.077
DURATION = 30 # number of years

# Additional Costs
# Home insurance 1500 per year
# HOA 150 per month
ADDITIONAL_MONTHLY_COSTS = 1500/12 + 150

# Tax
TAX_APPRAISAL_VAL = PRINCIPAL
TAX_APR = 0.02
TAX_APPRAISAL_VAL_GROWTH_RATE = 0.0

# Income
MONTHLY_INCOME = 200000/12 # Gross monthly income because debt-to-income ratio is calculated from gross

# Randomized Interest Rates
INT_RANGE_LOW = 0.02
INT_RANGE_HI = 0.11
INT_CHANGE_PERIOD_YEARS = 3 # randomize each number of years

# Don't touch
PAYMENTS_N = 12 # monthly payments. CAUTION - this should probably never be changed because I'm sure I've hardcoded 12 elsewhere
loan_value  = PRINCIPAL - DOWN_PAYMENT

# Property Tax

<span style="color:red">
</span>

In [43]:
tax_df = dataframes.property_tax_amortization(TAX_APPRAISAL_VAL, TAX_APR, DURATION, TAX_APPRAISAL_VAL_GROWTH_RATE)
# display(tax_df.round(0))

# Mortgage Amortization

In [44]:
ma_df = dataframes.mortgage_amortization(loan_value, LOAN_APR, PAYMENTS_N, DURATION, DOWN_PAYMENT)
display(ma_df.round(0))

fig = px.bar(ma_df, x=ma_df.index, y=['interest paid', 'principal paid'])
fig.add_scatter(x=ma_df.index, y=ma_df['mortgage balance'], name='mortgage balance')
fig.add_scatter(x=ma_df.index, y=ma_df['equity'], name='equity')
fig.show()

,mortgage balance,mortgage payment,payments,equity,principal paid,interest paid
month,,,,,,
0,700000.0,0.0,0.0,200000.0,0.0,0.0
1,699501.0,4991.0,4991.0,200499.0,499.0,4492.0
2,698999.0,4991.0,9981.0,201001.0,1001.0,8980.0
3,698493.0,4991.0,14972.0,201507.0,1507.0,13465.0
4,697984.0,4991.0,19963.0,202016.0,2016.0,17947.0
...,...,...,...,...,...,...
356,19647.0,4991.0,1776697.0,880353.0,680353.0,1096343.0
357,14782.0,4991.0,1781687.0,885218.0,685218.0,1096470.0
358,9886.0,4991.0,1786678.0,890114.0,690114.0,1096564.0


# Payment Amortization

In [45]:
as_df = dataframes.amortization_summary(ma_df, tax_df, monthly_income=MONTHLY_INCOME, monthly_additional_costs=ADDITIONAL_MONTHLY_COSTS)
display(as_df.round(0))

#Sums
fig = px.bar(as_df, x=as_df.index, y=['principal paid', 'interest paid', 'tax paid'], title='Amounts Paid')
fig.add_scatter(x=as_df.index, y=as_df['mortgage balance'], name='mortgage balance', mode='lines')
fig.add_scatter(x=as_df.index, y=as_df['equity'], name='equity', mode='lines')
fig.add_scatter(x=as_df.index, y=as_df['payments'], name='payments', mode='lines')
fig.add_scatter(x=as_df.index, y=as_df['appraisal value'], name='appraisal value', mode='lines')
fig.show()


Index(['mortgage balance', 'mortgage payment', 'payments', 'equity',
       'principal paid', 'interest paid', 'year'],
      dtype='object')


,mortgage balance,mortgage payment,payments,equity,principal paid,interest paid,year,monthly tax payment,appraisal value,monthly payment,tax paid,mortgage payment principal,mortgage payment interest,monthly income,debt to income ratio
month,,,,,,,,,,,,,,,
0,700000.0,0.0,NaN,200000.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,16667.0,NaN
1,699501.0,4991.0,6766.0,200499.0,499.0,4492.0,1,1500.0,900000.0,6766.0,1500.0,499.0,4492.0,16667.0,41.0
2,698999.0,4991.0,13531.0,201001.0,1001.0,8980.0,1,1500.0,900000.0,6766.0,3000.0,501.0,4490.0,16667.0,41.0
3,698493.0,4991.0,20297.0,201507.0,1507.0,13465.0,1,1500.0,900000.0,6766.0,4500.0,502.0,4488.0,16667.0,41.0
4,697984.0,4991.0,27063.0,202016.0,2016.0,17947.0,1,1500.0,900000.0,6766.0,6000.0,504.0,4487.0,16667.0,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,19647.0,4991.0,2408597.0,880353.0,680353.0,1096343.0,30,1500.0,900000.0,6766.0,534000.0,1911.0,3080.0,16667.0,41.0
357,14782.0,4991.0,2415362.0,885218.0,685218.0,1096470.0,30,1500.0,900000.0,6766.0,535500.0,1919.0,3071.0,16667.0,41.0
358,9886.0,4991.0,2422128.0,890114.0,690114.0,1096564.0,30,1500.0,900000.0,6766.0,537000.0,1928.0,3063.0,16667.0,41.0


# Monthly Payments

In [46]:
fig = px.bar(as_df, x=as_df.index, y=['mortgage payment principal', 'mortgage payment interest', 'monthly tax payment'], title='Monthly Payments')
fig.add_scatter(x=as_df.index, y=as_df['monthly payment'], name='monthly payment', mode='lines')
fig.show()

# Debt to Income Ratio

In [47]:
fig = px.scatter(as_df, x=as_df.index, y=['debt to income ratio'], title='Debt to Income Ratio')
fig.update_yaxes(range=[0, 100])
fig.show()

# Randomized Interest Rate
Randomize the interest rate every few years. Re-finance the loan into this new rate regardless of whether it's higher or lower.

In [48]:
ri_df, periodic_interest_rates = dataframes.randomized_interest(LOAN_APR, INT_RANGE_LOW, INT_RANGE_HI, INT_CHANGE_PERIOD_YEARS, DURATION * PAYMENTS_N)
fig = px.scatter(ri_df, x=ri_df.index, y=['interest rate'], title='Randomized Interest Rate')
fig.show()
#print(periodic_interest_rates)

# comment this out to not debug
# periodic_interest_rates = [0.077, 0.07309579158741138, 0.046674635865706436, 0.10835080679897587, 0.0968527302624414, 0.08129664568334291, 0.10207750869120687, 0.08102664288489458, 0.08718379322627176, 0.04845177074687092]
# periodic_interest_rates = [LOAN_APR for i in range(DURATION // INT_CHANGE_PERIOD_YEARS)]

ma_df = dataframes.refi_mortgage_amortization(PRINCIPAL, PAYMENTS_N, DURATION, DOWN_PAYMENT, INT_CHANGE_PERIOD_YEARS, periodic_interest_rates)

# debug remove me
for i in range(len(periodic_interest_rates)):
  start = i*PAYMENTS_N*INT_CHANGE_PERIOD_YEARS
  end = start + 2
  display(ma_df[start:end].round(0))
display(ma_df[360:361].round(0))

# display(ma_df.round(0))

fig = px.bar(ma_df, x=ma_df.index, y=['interest paid', 'principal paid'])
fig.add_scatter(x=ma_df.index, y=ma_df['mortgage balance'], name='mortgage balance')
fig.add_scatter(x=ma_df.index, y=ma_df['equity'], name='equity')
fig.show()

ma_df for apr = 0.077, duration = 30
ma_df for apr = 0.077, duration = 27
ma_df for apr = 0.077, duration = 24
ma_df for apr = 0.077, duration = 21
ma_df for apr = 0.077, duration = 18
ma_df for apr = 0.077, duration = 15
ma_df for apr = 0.077, duration = 12
ma_df for apr = 0.077, duration = 9
ma_df for apr = 0.077, duration = 6
ma_df for apr = 0.077, duration = 3


,mortgage balance,mortgage payment,payments,equity,principal paid,interest paid,interest rate %
month,,,,,,,
0,700000.0,0.0,0.0,200000.0,0.0,0.0,8.0
1,699501.0,4991.0,4991.0,200499.0,499.0,4492.0,8.0


,mortgage balance,mortgage payment,payments,equity,principal paid,interest paid,interest rate %
month,,,,,,,
36,679862.0,4991.0,179666.0,220138.0,20138.0,159528.0,8.0
37,679234.0,4991.0,184657.0,220766.0,20766.0,163890.0,8.0


,mortgage balance,mortgage payment,payments,equity,principal paid,interest paid,interest rate %
month,,,,,,,
72,654509.0,4991.0,359332.0,245491.0,45491.0,313841.0,8.0
73,653718.0,4991.0,364323.0,246282.0,46282.0,318041.0,8.0


,mortgage balance,mortgage payment,payments,equity,principal paid,interest paid,interest rate %
month,,,,,,,
108,622592.0,4991.0,538998.0,277408.0,77408.0,461590.0,8.0
109,621596.0,4991.0,543989.0,278404.0,78404.0,465585.0,8.0


...

,mortgage balance,mortgage payment,payments,equity,principal paid,interest paid,interest rate %
month,,,,,,,
216,468141.0,4991.0,1077996.0,431859.0,231859.0,846137.0,8.0
217,466155.0,4991.0,1082987.0,433845.0,233845.0,849141.0,8.0


,mortgage balance,mortgage payment,payments,equity,principal paid,interest paid,interest rate %
month,,,,,,,
252,387968.0,4991.0,1257662.0,512032.0,312032.0,945630.0,8.0
253,385467.0,4991.0,1262652.0,514533.0,314533.0,948119.0,8.0


,mortgage balance,mortgage payment,payments,equity,principal paid,interest paid,interest rate %
month,,,,,,,
288,287036.0,4991.0,1437328.0,612964.0,412964.0,1024363.0,8.0
289,283887.0,4991.0,1442318.0,616113.0,416113.0,1026205.0,8.0


,mortgage balance,mortgage payment,payments,equity,principal paid,interest paid,interest rate %
month,,,,,,,
324,159968.0,4991.0,1616994.0,740032.0,540032.0,1076962.0,8.0
325,156004.0,4991.0,1621984.0,743996.0,543996.0,1077989.0,8.0


,mortgage balance,mortgage payment,payments,equity,principal paid,interest paid,interest rate %
month,,,,,,,
360,0.0,4991.0,1796660.0,900000.0,700000.0,1096660.0,8.0
